In [1]:
import os
import numpy as np
import glob,re

# Routines

In [2]:
def Calc_n(omega,reG,T,points=20):

    omegal=omega[-points:]
    reGl=reG[-points:]

    def func(t,a,b):

            return a/(t**2+b**2)
    x=omegal
    y=reGl

    from scipy.optimize import curve_fit
    popt, pcov = curve_fit(func, x, y)
    
    A=popt[0]
    C=popt[1]
    
    #print A,C
    
    n=0
    
    n+=np.sum(reG)*2*T
    #print "sumre: ",n
    n+=A/C*(np.cosh(C/T)/np.sinh(C/T)-0.5*np.cosh(C/T/2)/np.sinh(C/T/2))-T*np.sum(2*A/(omega**2+C**2))
    #print "+tail: ",n
    n+=0.5
    
    return n

class mu_bisection:
    def __init__(self,mu_left,mu_right,dmu = 0.1,tolr = 1e-3,max_iters = 7):
        if mu_left > mu_right: #order is wrong, exchange !
            self.muL = mu_right
            self.muR = mu_left
            self.mu_exchanged = True
        else: #order is right
            self.muL = mu_left
            self.muR = mu_right
            self.mu_exchanged = False
        self.dmu = abs(dmu)
        self.iters = 0
        self.tolr = abs(tolr)
        self.max_iters = max_iters
        self.left_found = False
        self.right_found = False
        self.right_OK = False
        self.converged = False
        self.message = "mu_bisection is initialized!"
        self.mu = None
        self.mu_last = None
        self.mu_last_last = None
        self.N = None
        self.N_last = None
        self.N_last_last = None
        self.N_left = None
        self.N_right = None
        self.Nbisection = 0
        self.Nsecant = 0
        self.tries = 0
        self.secant_failed = False
        
    def run(self,N_func):
        
        # First check if (muR+muL)/2 is good enough or not (this minimizes effort if guess is good)
        print "Checking midpoint..."
        self.muM = (self.muL+self.muR)*0.5
        self.N  = N_func((self.muL+self.muR)*0.5)
        self.N_diff = self.N-0.5
        if abs(self.N_diff) < self.tolr:
            self.converged = True
            self.message = "Solution converged: N_diff = %le, N = %f"%(self.N_diff,self.N)
            return self.mu
        elif self.N > 0.5-self.tolr:
            #Set muM as muR
            self.N_right = self.N
            self.muR = self.muM
            # CHECK FOR muL
            print "Checking if guess of muL is good..."
            self.N_left = N_func(self.muL) 
            self.N_diff = self.N_left-0.5
            if abs(self.N_diff) < self.tolr:
                self.converged = True
                self.message = "Solution converged: N_diff = %le, N = %f"%(self.N_diff,self.N)
                return self.muL
            while (self.N_left>0.5-self.tolr): #muL is too positive, muR is Okay
                print "muL is too positive, adjusting..."
                self.right_OK == True
                self.muR = self.muL #creep muR
                self.muL -= self.dmu #reduce muL
                self.N_left = N_func(self.muL) 
                self.N_diff = self.N_left-0.5
                if abs(self.N_diff) < self.tolr:
                    self.converged = True
                    self.message = "Solution converged: N_diff = %le, N = %f"%(self.N_diff,self.N)
                    return self.muL
                self.tries += 1
                if self.tries > self.max_iters:
                    return None
            self.left_found == True
            self.N = self.N_left
            print "muL is good!"
        else:
            #Set muM as muL
            self.N_left = self.N
            self.muL = self.muM
            # CHECK FOR muR
            print "Checking if guess of muR is good..."
            #if self.right_OK == False: #muR is not checked
            self.N_right = N_func(self.muR)
            self.N_diff = self.N_right-0.5
            if abs(self.N_diff) < self.tolr:
                self.converged = True
                self.message = "Solution converged: N_diff = %le, N = %f"%(self.N_diff,self.N)
                return self.muR
            while (self.N_right<0.5+self.tolr): #muR is too negative
                print "muR is too negative, adjusting..."
                self.muL = self.muR #creep muL
                self.muR += self.dmu #increased muR
                self.N_right = N_func(self.muR) 
                self.N_diff = self.N_right-0.5
                if abs(self.N_diff) < self.tolr:
                    self.converged = True
                    self.message = "Solution converged: N_diff = %le, N = %f"%(self.N_diff,self.N)
                    return self.muR
                self.tries += 1
                if self.tries > self.max_iters:
                    return None
            self.right_found == True
            self.N = self.N_right
            print "muR is good!"
        
        self.N_diff = self.N-0.5
        self.mu_diff = self.muR-self.muL
        
        #Use interpolation to obtain a guess of mu
        #Perform the bisection
        #while (self.mu_diff > self.tolr and self.iters < self.max_iters):
        while (abs(self.N_diff) > self.tolr and self.iters < self.max_iters):
            
            #roll values
            self.N_last_last = self.N_last 
            self.mu_last_last = self.mu_last
            self.N_last = self.N
            self.mu_last = self.mu
            
            #initial run, use muR and muL
            if self.N_last == None or self.N_last_last == None: 
                self.N_last  = self.N_left
                self.N_last_last = self.N_right
                self.mu_last = self.muL
                self.mu_last_last = self.muR
            
            #Use secant method to calculate, if failed to quickly converge, fall back to bisection
            if not(self.secant_failed):
                print "Secant run"
                self.mu = (0.5-self.N_left)/(self.N_right-self.N_left)*(self.muR-self.muL)+self.muL
                if not(self.mu<self.muR and self.mu > self.muL): #Fall back to bisection
                    self.Nbisection += 1
                    self.mu = 0.5*self.muL+0.5*self.muR
                else:
                    self.Nsecant += 1
                    self.secant_failed = True #Use bisection after this
            else:
                print "Bisection run"
                self.Nbisection += 1
                self.mu = 0.5*self.muL+0.5*self.muR
                self.secant_failed = False #try secant again
                 
            self.N = N_func(self.mu)
            if self.N > 0.5: #Shrink muR
                self.muR = self.mu
                self.N_right = self.N
            else:
                self.muL = self.mu
                self.N_left = self.N
            self.mu_diff = self.muR-self.muL #calculate new difference
            self.iters += 1
            self.N_diff = self.N - 0.5
            print "N_diff = %le"%(abs(self.N_diff))
            
        if abs(self.N_diff) < self.tolr:
            self.converged = True
            self.message = "Solution converged: N_diff = %le, N = %f, Nsecant = %d, Nbisection = %d"%(abs(self.N_diff),self.N,self.Nsecant,self.Nbisection)
            return self.mu
        else:
            self.converged = False
            self.message = "Maximum iterations reached: N_diff = %le, N = %f, Nsecant = %d, Nbisection = %d"%(abs(self.N_diff),self.N,self.Nsecant,self.Nbisection)
            return self.mu

# Run (Metal)

In [22]:
#Model Parameters
T=0.01
V=0.217
ef=-1
t=0.6

typ = 'Metal'

#initial guess for bounds of mu (can be a little generous!)
muR = -0.8
muL = -0.9

#step to search for mu-bounds if the initial guess is off
dmu = 0.1

#tolerance for ntot (This should be above Monte Carlo noise)
N_tolr = 1e-3

def run_impurity_solver(mu):
    
    #Path to save stuffs
    SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)
    
    #Run flags
    FLAGS="--cmix=0.95 --Niter=50 --Msteps=3e6 --NC=55 --mode=fromNI --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

    #Run Impurity Solver
    #cmd = "rm initialcondition/*"
    #os.system(cmd)
    cmd = "rm run/*"
    os.system(cmd)
    cmd = "cp initialcondition/Gc.out run/Gc.out"
    os.system(cmd)
    cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
    os.system(cmd)
        
    #Calc filling
    fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.4[1-9]"))
    #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
    nfs = []
    for fGf in fGfs:
        omega,reGf,imGf = np.loadtxt(fGf).T
        nf = Calc_n(omega,reGf,T,points=20)
        nfs.append(nf)

    fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.4[1-9]"))
    #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
    ncs = []
    for fGc in fGcs:
        omega,reGc,imGc = np.loadtxt(fGc).T
        nc = Calc_n(omega,reGc,T,points=20)
        ncs.append(nc)

    nfavg = np.average(np.array(nfs))
    ncavg = np.average(np.array(ncs))
    nfstd = np.std(np.array(nfs))
    ncstd = np.std(np.array(ncs))

    ntot = ncavg+nfavg
    ntotstd = nfstd+ncstd

    print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)

    fn = os.path.join("results/%s"%SPATH,"occupation.out")
    f = open(fn, 'w')
    print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
    print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
    f.close()
    
    return ntot

#initialize bisection search
search = mu_bisection(muL,muR,dmu = dmu,tolr = N_tolr)
mu = search.run(run_impurity_solver)

print search.message

#Create soft link to quarter-filled case
lnpath="results/CT/T.%.04f/%s/t.%.04f/converged"%(T,typ,t)
tolink = os.path.join(os.getcwd(),"results/%s"%SPATH)
cmd = "ln -s -f %s %s"%(tolink,lnpath)
os.system(cmd)

Checking midpoint...
mu = -0.850000 , ntot(avg) = 0.546000 
 nf(avg) = 0.446450 , nc(avg) = 0.099550
Checking if guess of muL is good...
mu = -0.900000 , ntot(avg) = 0.512702 
 nf(avg) = 0.431525 , nc(avg) = 0.081177
muL is too positive, adjusting...
mu = -1.000000 , ntot(avg) = 0.407515 
 nf(avg) = 0.359319 , nc(avg) = 0.048196
muL is good!
Secant run
mu = -0.933217 , ntot(avg) = 0.486235 
 nf(avg) = 0.416599 , nc(avg) = 0.069637
N_diff = 1.376471e-02
Bisection run
mu = -0.916608 , ntot(avg) = 0.499950 
 nf(avg) = 0.424616 , nc(avg) = 0.075335
N_diff = 4.979165e-05
Solution converged: N_diff = 4.979165e-05, N = 0.499950, Nsecant = 1, Nbisection = 1


In [14]:
#Model Parameters
T=0.01
V=0.217
ef=-1
t=0.35

typ = 'Metal'

#initial guess for bounds of mu (can be a little generous!)
mu0 = 0.242702-1
muR = mu0+0.05
muL = mu0-0.05

#step to search for mu-bounds if the initial guess is off
dmu = 0.1

#tolerance for ntot (This should be above Monte Carlo noise)
N_tolr = 1e-3

def run_impurity_solver(mu):
    
    #Path to save stuffs
    SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)
    
    #Run flags
    FLAGS="--cmix=0.95 --Niter=50 --Msteps=3e6 --NC=55 --mode=fromIC --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

    #Run Impurity Solver
    #cmd = "rm initialcondition/*"
    #os.system(cmd)
    cmd = "rm run/*"
    os.system(cmd)
    cmd = "cp initialcondition/Gc.out run/Gc.out"
    os.system(cmd)
    cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
    os.system(cmd)
        
    #Calc filling
    fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.4[1-9]"))
    #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
    nfs = []
    for fGf in fGfs:
        omega,reGf,imGf = np.loadtxt(fGf).T
        nf = Calc_n(omega,reGf,T,points=20)
        nfs.append(nf)

    fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.4[1-9]"))
    #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
    ncs = []
    for fGc in fGcs:
        omega,reGc,imGc = np.loadtxt(fGc).T
        nc = Calc_n(omega,reGc,T,points=20)
        ncs.append(nc)

    nfavg = np.average(np.array(nfs))
    ncavg = np.average(np.array(ncs))
    nfstd = np.std(np.array(nfs))
    ncstd = np.std(np.array(ncs))

    ntot = ncavg+nfavg
    ntotstd = nfstd+ncstd

    print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)

    fn = os.path.join("results/%s"%SPATH,"occupation.out")
    f = open(fn, 'w')
    print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
    print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
    f.close()
    
    return ntot

#initialize bisection search
search = mu_bisection(muL,muR,dmu = dmu,tolr = N_tolr)
mu = search.run(run_impurity_solver)

print search.message

#Create soft link to quarter-filled case
lnpath="results/CT/T.%.04f/%s/t.%.04f/converged"%(T,typ,t)
tolink = os.path.join(os.getcwd(),"results/%s"%SPATH)
cmd = "ln -s -f %s %s"%(tolink,lnpath)
os.system(cmd)

Checking midpoint...


/opt/intel/intelpython2/lib/python2.7/site-packages/scipy/optimize/minpack.py:787: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


mu = -0.757298 , ntot(avg) = 0.479562 
 nf(avg) = 0.468379 , nc(avg) = 0.011182
Checking if guess of muR is good...
mu = -0.707298 , ntot(avg) = 0.494275 
 nf(avg) = 0.469677 , nc(avg) = 0.024598
muR is too negative, adjusting...
mu = -0.607298 , ntot(avg) = 0.539014 
 nf(avg) = 0.472870 , nc(avg) = 0.066145
muR is good!
Secant run
mu = -0.672921 , ntot(avg) = 0.509827 
 nf(avg) = 0.472114 , nc(avg) = 0.037713
N_diff = 9.827077e-03
Bisection run
mu = -0.690109 , ntot(avg) = 0.501368 
 nf(avg) = 0.470458 , nc(avg) = 0.030910
N_diff = 1.368337e-03
Secant run
mu = -0.691188 , ntot(avg) = 0.501392 
 nf(avg) = 0.470975 , nc(avg) = 0.030417
N_diff = 1.392017e-03
Bisection run
mu = -0.699243 , ntot(avg) = 0.499116 
 nf(avg) = 0.471770 , nc(avg) = 0.027346
N_diff = 8.837297e-04
Solution converged: N_diff = 8.837297e-04, N = 0.499116, Nsecant = 2, Nbisection = 2


0

In [23]:
T=0.01
typ = 'Metal'
t=0.3

#Path to save stuffs
SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,typ,t)

#Read in mu
f = open(os.path.join("results/",SPATH,"PARAMS"), "r")
contents = f. read()
f. close()
print contents
char = re.findall(r"mu [-+]?\d+\.\d+", contents)
mu = float(char[0][3:])+ef

print mu

# Input file for continuous time quantum Monte Carlo
nom 80 	# number of Matsubara frequency points to sample
svd_lmax 30 	# number of SVD functions to project the solution
tsample 200 	# how often to record the measurements
aom 1 	# number of frequency points to determin high frequency tail
M 20000000.0 	# Number of Monte Carlo steps
beta 100.0 	# Inverse temperature
U 2.0 	# Coulomb repulsion (F0)
GlobalFlip 1000000 	# how often to perform global flip
exe mpirun -np 55 ctqmc 	# Path to executable
mu 0.348217 	# Chemical potential
mode SM 	# S stands for self-energy sampling, M stands for high frequency moment tail
Delta Delta.inp 	# Input bath function hybridization
cix one_band.imp 	# Input file with atomic state

-0.651783


## Scan (Metal)

In [ ]:
#Model Parameters
T=0.01
V=0.217
ef=-1

#COPY INITIAL CONDITION
typ = 'Metal'
t=0.3

cmd = "cp results/CT/T.%.04f/%s/t.%.04f/converged/Gc.out initialcondition/Gc.out"%(T,'Metal',t)
os.system(cmd)

#Path to save stuffs
SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,typ,t)

#Read in mu
f = open(os.path.join("results/",SPATH,"PARAMS"), "r")
contents = f. read()
f. close()
#print contents
char = re.findall(r"mu [-+]?\d+\.\d+", contents)
mu0 = float(char[0][3:])+ef

print mu0

for t in [0.25]:

    #step to search for mu-bounds if the initial guess is off
    dmu = 0.1
    
    #initial guess for bounds of mu (can be a little generous!)
    muR = mu0+dmu/2
    muL = -0.651783#mu0-dmu/2

    #tolerance for ntot (This should be above Monte Carlo noise)
    N_tolr = 1e-3

    def run_impurity_solver(mu):

        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)

        #Run flags
        FLAGS="--cmix=0.95 --Niter=30 --Msteps=5e6 --NC=55 --mode=fromIC --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

        #Run Impurity Solver
        #cmd = "rm initialcondition/*"
        #os.system(cmd)
        cmd = "rm run/*"
        os.system(cmd)
        cmd = "cp initialcondition/Gc.out run/Gc.out"
        os.system(cmd)
        cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
        os.system(cmd)
        
        
        #Calc filling
        fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.2[1-9]"))
        #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
        nfs = []
        for fGf in fGfs:
            omega,reGf,imGf = np.loadtxt(fGf).T
            nf = Calc_n(omega,reGf,T,points=20)
            nfs.append(nf)

        fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.2[1-9]"))
        #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
        ncs = []
        for fGc in fGcs:
            omega,reGc,imGc = np.loadtxt(fGc).T
            nc = Calc_n(omega,reGc,T,points=20)
            ncs.append(nc)

        nfavg = np.average(np.array(nfs))
        ncavg = np.average(np.array(ncs))
        nfstd = np.std(np.array(nfs))
        ncstd = np.std(np.array(ncs))
        
        ntot = ncavg+nfavg
        ntotstd = nfstd+ncstd

        print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)
        
        fn = os.path.join("results/%s"%SPATH,"occupation.out")
        f = open(fn, 'w')
        print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
        print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
        f.close()

        return ntot

    #initialize bisection search
    search = mu_bisection(muL,muR,dmu = dmu,tolr = N_tolr)
    mu = search.run(run_impurity_solver)

    print search.message
    

    
    #Adiabatic switching
    SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)
    fGc = os.path.join("results/%s"%SPATH,"Gc.out")
    cmd = "cp %s initialcondition/Gc.out"%fGc
    os.system(cmd)
    mu0 = mu
    
    #Create soft link to quarter-filled case
    lnpath="results/CT/T.%.04f/%s/t.%.04f/converged"%(T,typ,t)
    tolink = os.path.join(os.getcwd(),"results/%s"%SPATH)
    cmd = "ln -s -f %s %s"%(tolink,lnpath)
    os.system(cmd)

-0.651783
Checking midpoint...


## Scan (Insulator)

In [12]:
#Model Parameters
T=0.01
V=0.217
ef=-1

typ = 'Insul'

#mu0 = -0.7573

cmd = "cp results/CT/T.%.04f/%s/t.%.04f/converged/Gc.out initialcondition/Gc.out"%(T,'Metal',0.3)
os.system(cmd)

for t in [0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5]:

    #step to search for mu-bounds if the initial guess is off
    #dmu = 0.1
    
    #initial guess for bounds of mu (can be a little generous!)
    #muR = mu0+dmu/2
    #muL = mu0-dmu/2

    #tolerance for ntot (This should be above Monte Carlo noise)
    #N_tolr = 1e-3

    def run_impurity_solver():
        
        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,'Metal',t)
        
        #Read in mu
        f = open(os.path.join("results/",SPATH,"PARAMS"), "r")
        contents = f. read()
        f. close()
        char = re.findall(r"mu [-+]?\d+\.\d+", contents)
        mu = float(char[0][3:])+ef

        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)

        #Run flags
        FLAGS="--cmix=0.95 --Niter=50 --Msteps=2e7 --NC=55 --mode=fromIC --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

        #Run Impurity Solver
        #cmd = "rm initialcondition/*"
        #os.system(cmd)
        cmd = "rm run/*"
        os.system(cmd)
        cmd = "cp initialcondition/Gc.out run/Gc.out"
        os.system(cmd)
        cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
        os.system(cmd)
        
        
        #Calc filling
        fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.4[1-9]"))
        #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
        nfs = []
        for fGf in fGfs:
            omega,reGf,imGf = np.loadtxt(fGf).T
            nf = Calc_n(omega,reGf,T,points=20)
            nfs.append(nf)

        fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.4[1-9]"))
        #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
        ncs = []
        for fGc in fGcs:
            omega,reGc,imGc = np.loadtxt(fGc).T
            nc = Calc_n(omega,reGc,T,points=20)
            ncs.append(nc)

        nfavg = np.average(np.array(nfs))
        ncavg = np.average(np.array(ncs))
        nfstd = np.std(np.array(nfs))
        ncstd = np.std(np.array(ncs))
        
        ntot = ncavg+nfavg
        ntotstd = nfstd+ncstd

        print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)
        
        fn = os.path.join("results/%s"%SPATH,"occupation.out")
        f = open(fn, 'w')
        print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
        print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
        f.close()

        return ntot,mu

    #initialize bisection search
    #search = mu_bisection(muL,muR,dmu = dmu,tolr = N_tolr)
    #mu = search.run(run_impurity_solver)
    ntot,mu = run_impurity_solver()

    #print search.message
    
    #Adiabatic switching
    SPATH="CT/T.%.04f/%s/t.%.04f/mu.%.04f/"%(T,typ,t,mu)
    fGc = os.path.join("results/%s"%SPATH,"Gc.out")
    cmd = "cp %s initialcondition/Gc.out"%fGc
    os.system(cmd)
    #mu0 = mu

    #Create soft link to quarter-filled case
    lnpath="results/CT/T.%.04f/%s/t.%.04f/converged"%(T,typ,t)

    tolink = os.path.join(os.getcwd(),"results/%s"%SPATH)
    cmd = "ln -s -f %s %s"%(tolink,lnpath)
    os.system(cmd)

mu = -0.767574 , ntot(avg) = 0.500126 
 nf(avg) = 0.464533 , nc(avg) = 0.035593
mu = -0.778266 , ntot(avg) = 0.500058 
 nf(avg) = 0.463219 , nc(avg) = 0.036839
mu = -0.789855 , ntot(avg) = 0.500148 
 nf(avg) = 0.462336 , nc(avg) = 0.037811
mu = -0.800572 , ntot(avg) = 0.499639 
 nf(avg) = 0.460471 , nc(avg) = 0.039168
mu = -0.811096 , ntot(avg) = 0.499586 
 nf(avg) = 0.458969 , nc(avg) = 0.040617
mu = -0.820799 , ntot(avg) = 0.499969 
 nf(avg) = 0.457586 , nc(avg) = 0.042383
mu = -0.831089 , ntot(avg) = 0.499552 
 nf(avg) = 0.455587 , nc(avg) = 0.043966
mu = -0.839425 , ntot(avg) = 0.499957 
 nf(avg) = 0.453706 , nc(avg) = 0.046250
mu = -0.850000 , ntot(avg) = 0.499546 
 nf(avg) = 0.451811 , nc(avg) = 0.047735


## Refine (Metal)

In [20]:
#Model Parameters
T=0.01
V=0.217
ef=-1

typ = 'Metal'


for t in [0.5]:

    def run_impurity_solver():

        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,typ,t)
        
        #Read in mu
        f = open(os.path.join("results/",SPATH,"PARAMS"), "r")
        contents = f. read()
        f. close()
        char = re.findall(r"mu [-+]?\d+\.\d+", contents)
        mu = float(char[0][3:])+ef

        #Run flags
        FLAGS="--cmix=0.95 --Niter=10 --Msteps=2e7 --NC=55 --mode=refine --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

        #Run Impurity Solver
        #cmd = "rm initialcondition/*"
        #os.system(cmd)
        cmd = "rm run/*"
        os.system(cmd)
        cmd = "cp initialcondition/Gc.out run/Gc.out"
        os.system(cmd)
        cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
        os.system(cmd)
        
        
        #Calc filling
        fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.6[5-9]"))
        #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
        nfs = []
        for fGf in fGfs:
            omega,reGf,imGf = np.loadtxt(fGf).T
            nf = Calc_n(omega,reGf,T,points=20)
            nfs.append(nf)

        fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.6[5-9]"))
        #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
        ncs = []
        for fGc in fGcs:
            omega,reGc,imGc = np.loadtxt(fGc).T
            nc = Calc_n(omega,reGc,T,points=20)
            ncs.append(nc)

        nfavg = np.average(np.array(nfs))
        ncavg = np.average(np.array(ncs))
        nfstd = np.std(np.array(nfs))
        ncstd = np.std(np.array(ncs))
        
        ntot = ncavg+nfavg
        ntotstd = nfstd+ncstd

        print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)
        
        fn = os.path.join("results/%s"%SPATH,"occupation.out")
        f = open(fn, 'w')
        print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
        print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
        f.close()

        return ntot

    run_impurity_solver()

mu = -0.850000 , ntot(avg) = 0.499425 
 nf(avg) = 0.451686 , nc(avg) = 0.047739


In [21]:
#Model Parameters
T=0.01
V=0.217
ef=-1

typ = 'Metal'


for t in [0.35]:

    def run_impurity_solver():

        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,typ,t)
        
        #Read in mu
        f = open(os.path.join("results/",SPATH,"PARAMS"), "r")
        contents = f. read()
        f. close()
        char = re.findall(r"mu [-+]?\d+\.\d+", contents)
        mu = float(char[0][3:])+ef
        #mu=-0.699243
        #Run flags
        FLAGS="--cmix=0.95 --Niter=10 --Msteps=2e7 --NC=55 --mode=refine --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

        #Run Impurity Solver
        #cmd = "rm initialcondition/*"
        #os.system(cmd)
        cmd = "rm run/*"
        os.system(cmd)
        cmd = "cp initialcondition/Gc.out run/Gc.out"
        os.system(cmd)
        cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
        os.system(cmd)
        
        
        #Calc filling
        fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.5[5-9]"))
        #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
        nfs = []
        for fGf in fGfs:
            omega,reGf,imGf = np.loadtxt(fGf).T
            nf = Calc_n(omega,reGf,T,points=20)
            nfs.append(nf)

        fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.5[5-9]"))
        #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
        ncs = []
        for fGc in fGcs:
            omega,reGc,imGc = np.loadtxt(fGc).T
            nc = Calc_n(omega,reGc,T,points=20)
            ncs.append(nc)

        nfavg = np.average(np.array(nfs))
        ncavg = np.average(np.array(ncs))
        nfstd = np.std(np.array(nfs))
        ncstd = np.std(np.array(ncs))
        
        ntot = ncavg+nfavg
        ntotstd = nfstd+ncstd

        print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)
        
        fn = os.path.join("results/%s"%SPATH,"occupation.out")
        f = open(fn, 'w')
        print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
        print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
        f.close()

        return ntot

    run_impurity_solver()

mu = -0.699243 , ntot(avg) = 0.498050 
 nf(avg) = 0.470597 , nc(avg) = 0.027453


# Input file for continuous time quantum Monte Carlo
nom 80 	# number of Matsubara frequency points to sample
svd_lmax 30 	# number of SVD functions to project the solution
tsample 200 	# how often to record the measurements
aom 1 	# number of frequency points to determin high frequency tail
M 20000000.0 	# Number of Monte Carlo steps
beta 100.0 	# Inverse temperature
U 2.0 	# Coulomb repulsion (F0)
GlobalFlip 1000000 	# how often to perform global flip
exe mpirun -np 55 ctqmc 	# Path to executable
mu 0.30075700000000005 	# Chemical potential
mode SM 	# S stands for self-energy sampling, M stands for high frequency moment tail
Delta Delta.inp 	# Input bath function hybridization
cix one_band.imp 	# Input file with atomic state

-0.699243


## Refine (Insulator)

In [ ]:
#Model Parameters
T=0.01
V=0.217
ef=-1

typ = 'Insul'


for t in [0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5]:

    def run_impurity_solver():

        #Path to save stuffs
        SPATH="CT/T.%.04f/%s/t.%.04f/converged/"%(T,typ,t)
        
        #Read in mu
        f = open(os.path.join(SPATH,"PARAMS"), "r")
        contents = f. read()
        f. close()
        char = re.findall(r"mu [-+]?\d+\.\d+", contents)
        mu = float(char[0][3:])+ef

        #Run flags
        FLAGS="--cmix=0.95 --Niter=50 --Msteps=1e7 --NC=55 --mode=refine --tsample=200 --nom=80 --adiabatic=False --gridsize=2000 --U=2.0"

        #Run Impurity Solver
        #cmd = "rm initialcondition/*"
        #os.system(cmd)
        cmd = "rm run/*"
        os.system(cmd)
        cmd = "cp initialcondition/Gc.out run/Gc.out"
        os.system(cmd)
        cmd = "./py_scripts/CT.py --mu=%f --T=%f --ef=%f --V=%f --t=%f --spath=%s %s"%(mu,T,ef,V,t,SPATH,FLAGS)
        os.system(cmd)
        
        
        #Calc filling
        fGfs = glob.glob(os.path.join("results",SPATH,"Gf.out.4[1-9]"))
        #fGf = os.path.join("results/%s"%SPATH,"Gf.out")
        nfs = []
        for fGf in fGfs:
            omega,reGf,imGf = np.loadtxt(fGf).T
            nf = Calc_n(omega,reGf,T,points=20)
            nfs.append(nf)

        fGcs = glob.glob(os.path.join("results",SPATH,"Gc.out.4[1-9]"))
        #fGc = os.path.join("results/%s"%SPATH,"Gc.out")
        ncs = []
        for fGc in fGcs:
            omega,reGc,imGc = np.loadtxt(fGc).T
            nc = Calc_n(omega,reGc,T,points=20)
            ncs.append(nc)

        nfavg = np.average(np.array(nfs))
        ncavg = np.average(np.array(ncs))
        nfstd = np.std(np.array(nfs))
        ncstd = np.std(np.array(ncs))
        
        ntot = ncavg+nfavg
        ntotstd = nfstd+ncstd

        print "mu = %f , ntot(avg) = %f \n nf(avg) = %f , nc(avg) = %f"%(mu,ntot,nfavg,ncavg)
        
        fn = os.path.join("results/%s"%SPATH,"occupation.out")
        f = open(fn, 'w')
        print >> f, "ntot(avg) = %f , nf(avg) = %f , nc(avg) = %f"%(ntot,nf,nc)
        print >> f, "ntot(std) = %f , nf(std) = %f , nc(std) = %f"%(ntotstd,nfstd,ncstd)
        f.close()

        return ntot

    run_impurity_solver()